# Εκπαίδευση Ενισχυτικής Μάθησης (RL) για Ισορροπία Cartpole

Αυτό το σημειωματάριο αποτελεί μέρος του [Προγράμματος Σπουδών AI για Αρχάριους](http://aka.ms/ai-beginners). Έχει εμπνευστεί από το [επίσημο tutorial του PyTorch](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) και από αυτήν την [υλοποίηση Cartpole με PyTorch](https://github.com/yc930401/Actor-Critic-pytorch).

Σε αυτό το παράδειγμα, θα χρησιμοποιήσουμε Ενισχυτική Μάθηση (RL) για να εκπαιδεύσουμε ένα μοντέλο να ισορροπεί έναν πόλο πάνω σε ένα καρότσι που μπορεί να κινείται αριστερά και δεξιά σε οριζόντια κλίμακα. Θα χρησιμοποιήσουμε το περιβάλλον [OpenAI Gym](https://www.gymlibrary.ml/) για να προσομοιώσουμε τον πόλο.

> **Σημείωση**: Μπορείτε να εκτελέσετε τον κώδικα αυτού του μαθήματος τοπικά (π.χ. από το Visual Studio Code), οπότε η προσομοίωση θα ανοίξει σε νέο παράθυρο. Όταν εκτελείτε τον κώδικα online, ίσως χρειαστεί να κάνετε κάποιες προσαρμογές στον κώδικα, όπως περιγράφεται [εδώ](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7).

Θα ξεκινήσουμε διασφαλίζοντας ότι το Gym είναι εγκατεστημένο:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Τώρα ας δημιουργήσουμε το περιβάλλον CartPole και ας δούμε πώς να το χειριστούμε. Ένα περιβάλλον έχει τις εξής ιδιότητες:

* **Χώρος ενεργειών** είναι το σύνολο των πιθανών ενεργειών που μπορούμε να εκτελέσουμε σε κάθε βήμα της προσομοίωσης  
* **Χώρος παρατηρήσεων** είναι ο χώρος των παρατηρήσεων που μπορούμε να κάνουμε  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Ας δούμε πώς λειτουργεί η προσομοίωση. Η παρακάτω επανάληψη εκτελεί την προσομοίωση, μέχρι η `env.step` να μην επιστρέφει τη σημαία τερματισμού `done`. Θα επιλέγουμε τυχαίες ενέργειες χρησιμοποιώντας το `env.action_space.sample()`, που σημαίνει ότι το πείραμα πιθανότατα θα αποτύχει πολύ γρήγορα (το περιβάλλον CartPole τερματίζεται όταν η ταχύτητα του CartPole, η θέση του ή η γωνία του βρίσκονται εκτός ορισμένων ορίων).

> Η προσομοίωση θα ανοίξει σε νέο παράθυρο. Μπορείτε να εκτελέσετε τον κώδικα πολλές φορές και να δείτε πώς συμπεριφέρεται.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Μπορείτε να παρατηρήσετε ότι οι παρατηρήσεις περιέχουν 4 αριθμούς. Αυτοί είναι:
- Θέση του καροτσιού
- Ταχύτητα του καροτσιού
- Γωνία του πόλου
- Ρυθμός περιστροφής του πόλου

`rew` είναι η ανταμοιβή που λαμβάνουμε σε κάθε βήμα. Μπορείτε να δείτε ότι στο περιβάλλον CartPole λαμβάνετε 1 πόντο για κάθε βήμα προσομοίωσης, και ο στόχος είναι να μεγιστοποιήσετε τη συνολική ανταμοιβή, δηλαδή τον χρόνο που το CartPole μπορεί να ισορροπήσει χωρίς να πέσει.

Κατά τη διάρκεια της ενισχυτικής μάθησης, ο στόχος μας είναι να εκπαιδεύσουμε μια **πολιτική** $\pi$, η οποία για κάθε κατάσταση $s$ θα μας λέει ποια ενέργεια $a$ να πάρουμε, δηλαδή ουσιαστικά $a = \pi(s)$.

Αν θέλετε μια πιθανή λύση, μπορείτε να σκεφτείτε την πολιτική ως επιστροφή ενός συνόλου πιθανοτήτων για κάθε ενέργεια, δηλαδή $\pi(a|s)$ θα σημαίνει την πιθανότητα να πάρουμε την ενέργεια $a$ στην κατάσταση $s$.

## Μέθοδος Πολιτικής Κλίσης

Στον πιο απλό αλγόριθμο RL, που ονομάζεται **Πολιτική Κλίσης**, θα εκπαιδεύσουμε ένα νευρωνικό δίκτυο να προβλέπει την επόμενη ενέργεια.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Θα εκπαιδεύσουμε το δίκτυο εκτελώντας πολλά πειράματα και ενημερώνοντας το δίκτυό μας μετά από κάθε εκτέλεση. Ας ορίσουμε μια συνάρτηση που θα εκτελεί το πείραμα και θα επιστρέφει τα αποτελέσματα (η λεγόμενη **ίχνος**) - όλες τις καταστάσεις, τις ενέργειες (και τις προτεινόμενες πιθανότητές τους) και τις ανταμοιβές:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Μπορείτε να εκτελέσετε ένα επεισόδιο με μη εκπαιδευμένο δίκτυο και να παρατηρήσετε ότι η συνολική ανταμοιβή (γνωστή και ως διάρκεια του επεισοδίου) είναι πολύ χαμηλή:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Ένα από τα δύσκολα σημεία του αλγορίθμου πολιτικής κλίσης είναι η χρήση **εκπτωτικών ανταμοιβών**. Η ιδέα είναι ότι υπολογίζουμε το διάνυσμα των συνολικών ανταμοιβών σε κάθε βήμα του παιχνιδιού, και κατά τη διάρκεια αυτής της διαδικασίας εκπτύσσουμε τις πρώιμες ανταμοιβές χρησιμοποιώντας κάποιον συντελεστή $gamma$. Επίσης, κανονικοποιούμε το προκύπτον διάνυσμα, επειδή θα το χρησιμοποιήσουμε ως βάρος για να επηρεάσουμε την εκπαίδευσή μας:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Τώρα ας ξεκινήσουμε την πραγματική εκπαίδευση! Θα τρέξουμε 300 επεισόδια, και σε κάθε επεισόδιο θα κάνουμε τα εξής:

1. Εκτελούμε το πείραμα και συλλέγουμε την ιχνηλάτηση.
1. Υπολογίζουμε τη διαφορά (`gradients`) μεταξύ των ενεργειών που πραγματοποιήθηκαν και των προβλεπόμενων πιθανοτήτων. Όσο μικρότερη είναι η διαφορά, τόσο πιο σίγουροι είμαστε ότι έχουμε πάρει τη σωστή ενέργεια.
1. Υπολογίζουμε τις προεξοφλημένες ανταμοιβές και πολλαπλασιάζουμε τα gradients με τις προεξοφλημένες ανταμοιβές - αυτό θα διασφαλίσει ότι τα βήματα με υψηλότερες ανταμοιβές θα έχουν μεγαλύτερη επίδραση στο τελικό αποτέλεσμα από αυτά με χαμηλότερες ανταμοιβές.
1. Οι αναμενόμενες ενέργειες-στόχοι για το νευρωνικό μας δίκτυο θα προέρχονται εν μέρει από τις προβλεπόμενες πιθανότητες κατά τη διάρκεια της εκτέλεσης και εν μέρει από τα υπολογισμένα gradients. Θα χρησιμοποιήσουμε την παράμετρο `alpha` για να καθορίσουμε σε ποιο βαθμό λαμβάνονται υπόψη τα gradients και οι ανταμοιβές - αυτό ονομάζεται *ρυθμός μάθησης* του αλγορίθμου ενίσχυσης.
1. Τέλος, εκπαιδεύουμε το δίκτυό μας στις καταστάσεις και τις αναμενόμενες ενέργειες, και επαναλαμβάνουμε τη διαδικασία.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Τώρα ας εκτελέσουμε το επεισόδιο με απόδοση για να δούμε το αποτέλεσμα:


In [ ]:
_ = run_episode(render=True)

Ελπίζουμε να βλέπετε ότι η ράβδος μπορεί πλέον να ισορροπεί αρκετά καλά!

## Μοντέλο Actor-Critic

Το μοντέλο Actor-Critic αποτελεί περαιτέρω εξέλιξη των policy gradients, στο οποίο δημιουργούμε ένα νευρωνικό δίκτυο για να μάθει τόσο την πολιτική όσο και τις εκτιμώμενες ανταμοιβές. Το δίκτυο θα έχει δύο εξόδους (ή μπορείτε να το δείτε ως δύο ξεχωριστά δίκτυα):
* **Actor** θα προτείνει την ενέργεια που πρέπει να ληφθεί, δίνοντάς μας την κατανομή πιθανοτήτων κατάστασης, όπως στο μοντέλο policy gradient.
* **Critic** θα εκτιμήσει ποια θα ήταν η ανταμοιβή από αυτές τις ενέργειες. Επιστρέφει τις συνολικές εκτιμώμενες ανταμοιβές στο μέλλον για τη δεδομένη κατάσταση.

Ας ορίσουμε ένα τέτοιο μοντέλο:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

Θα χρειαστεί να τροποποιήσουμε ελαφρώς τις συναρτήσεις `discounted_rewards` και `run_episode`:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Τώρα θα εκτελέσουμε τον κύριο βρόχο εκπαίδευσης. Θα χρησιμοποιήσουμε τη διαδικασία χειροκίνητης εκπαίδευσης του δικτύου υπολογίζοντας κατάλληλες συναρτήσεις απώλειας και ενημερώνοντας τις παραμέτρους του δικτύου:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Συμπέρασμα

Είδαμε δύο αλγόριθμους Ενισχυτικής Μάθησης (RL) σε αυτή την επίδειξη: τον απλό αλγόριθμο policy gradient και τον πιο εξελιγμένο actor-critic. Μπορείτε να παρατηρήσετε ότι αυτοί οι αλγόριθμοι λειτουργούν με αφηρημένες έννοιες όπως η κατάσταση, η δράση και η ανταμοιβή - γι' αυτό μπορούν να εφαρμοστούν σε πολύ διαφορετικά περιβάλλοντα.

Η Ενισχυτική Μάθηση μας επιτρέπει να μάθουμε την καλύτερη στρατηγική για την επίλυση ενός προβλήματος απλώς παρατηρώντας την τελική ανταμοιβή. Το γεγονός ότι δεν χρειαζόμαστε σύνολα δεδομένων με ετικέτες μας δίνει τη δυνατότητα να επαναλαμβάνουμε προσομοιώσεις πολλές φορές για να βελτιστοποιήσουμε τα μοντέλα μας. Παρ' όλα αυτά, υπάρχουν ακόμα πολλές προκλήσεις στην Ενισχυτική Μάθηση, τις οποίες μπορείτε να ανακαλύψετε αν αποφασίσετε να εστιάσετε περισσότερο σε αυτόν τον συναρπαστικό τομέα της Τεχνητής Νοημοσύνης.



---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
